In [1]:
import torch
import torch.nn.functional as F
from google.protobuf import text_format
from second.utils import simplevis
from second.pytorch.train import build_network
from second.protos import pipeline_pb2
from second.utils import config_tool
from second.pytorch.builder import (box_coder_builder, input_reader_builder,
                                    lr_scheduler_builder, optimizer_builder,
                                    second_builder)
from second.data.preprocess import merge_second_batch, merge_second_batch_multigpu
from second.pytorch.train import _worker_init_fn

from second.pytorch.train import example_convert_to_torch

from second.sphere.conv import init_depth_from_feature, depth_to_3D


In [13]:
# config_path = "/home/yy/deeplearning/deeplearning/mypackages/second/configs/car.lite.nb.config"
config_path = "/home/gx/GitHub/depconv/second/configs/car.fhd.config"
config = pipeline_pb2.TrainEvalPipelineConfig()
with open(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, config)
input_cfg = config.train_input_reader
eval_input_cfg = config.eval_input_reader
model_cfg = config.model.second
# config_tool.change_detection_range(model_cfg, [-50, -50, 50, 50])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


In [16]:
ckpt_path = "/home/gx/GitHub/depconv/second/depconv93/voxelnet-27900.tckpt"
net = build_network(model_cfg).to(device).eval()
net.load_state_dict(torch.load(ckpt_path))
target_assigner = net.target_assigner
voxel_generator = net.voxel_generator

dataset = input_reader_builder.build(
    input_cfg,
    model_cfg,
    training=True,
    voxel_generator=voxel_generator,
    target_assigner=target_assigner,
    multi_gpu=False)
eval_dataset = input_reader_builder.build(
        eval_input_cfg,
        model_cfg,
        training=False,
        voxel_generator=voxel_generator,
        target_assigner=target_assigner)

num_gpu=1
collate_fn= merge_second_batch
multi_gpu = False
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        shuffle=True,
#         num_workers=input_cfg.preprocess.num_workers * num_gpu,
        num_workers=1,
        pin_memory=False,
        collate_fn=collate_fn,
        worker_init_fn=_worker_init_fn,
        drop_last=not multi_gpu)

eval_dataloader = torch.utils.data.DataLoader(
        eval_dataset,
        batch_size=4,
        shuffle=False,
        num_workers=input_cfg.preprocess.num_workers,
        pin_memory=False,
        collate_fn=merge_second_batch)

voxelmap_shape =  (128, 512, 512)
dense_shape =  [1, 128, 512, 512, 16]
input sparse_shape =  [129 512 512]
rpn debug, final_num_filters =  384
rpn debug, num_cls =  2
rpn debug, num_anchor_per_loc =  2
{'Car': 5}
[-1]
load 2207 Pedestrian database infos
load 14357 Car database infos
load 734 Cyclist database infos
load 1297 Van database infos
load 488 Truck database infos
load 224 Tram database infos
load 337 Misc database infos
load 56 Person_sitting database infos
After filter database:
load 2207 Pedestrian database infos
load 13442 Car database infos
load 734 Cyclist database infos
load 1297 Van database infos
load 488 Truck database infos
load 224 Tram database infos
load 337 Misc database infos
load 56 Person_sitting database infos
feautres size [1, 128, 128]
res = (1, 128, 128, 1, 2, 7)
feautres size [1, 128, 128]
res = (1, 128, 128, 1, 2, 7)
remain number of infos: 3712
feautres size [1, 128, 128]
res = (1, 128, 128, 1, 2, 7)
feautres size [1, 128, 128]
res = (1, 128, 128, 1, 2

In [17]:
net

VoxelNet(
  (voxel_feature_extractor): SimpleVoxel()
  (middle_feature_extractor): SpMiddleFHD(
    (middle_conv): SparseSequential(
      (0): SubMConv3d()
      (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): SubMConv3d()
      (4): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): SparseConv3d()
      (7): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (8): ReLU()
      (9): SubMConv3d()
      (10): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (11): ReLU()
      (12): SubMConv3d()
      (13): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (14): ReLU()
      (15): SparseConv3d()
      (16): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (17): ReLU()
      (18): SubMConv3d()
      (19): BatchNorm1d(64

In [20]:
net.eval()
detections = []
for example in iter(eval_dataloader):
    example = example_convert_to_torch(example, torch.float32)
    with torch.no_grad():
        detections += net(example)
    
    result_dict = eval_dataset.dataset.evaluation(detections, "")

voxel_features shape:  torch.Size([71154, 4])
sparse_shape : [129 512 512]


RuntimeError: /home/gx/GitHub/depconv/spconv/src/spconv/indice.cu 193
cuda execution failed with error 2

torch.float32